In [ ]:
import csv
from cv2 import imread,cvtColor
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
file = open('driver_imgs_list.csv')
lst = csv.reader(file)
file_list = []
for i in lst:
    file_list.append(i)
file_list.pop(0)
w = 96
h = 128

In [ ]:
def read_file(file):
    img = cv2.imread('train/'+file[1]+'/'+file[2],0)
    img = cv2.resize(img,(w,h), interpolation=cv2.INTER_LINEAR)
    img = img.astype(np.float32)/255.0
    label = int(file[1][1])
    return img,label
def show(array):
    plt.figure()
    plt.imshow(array.reshape([h,w]), cmap="gray")
    plt.show()
def get_batch(batch_size):
    j = np.random.randint(0,20788,batch_size)
    batch = np.zeros((batch_size,h,w,1))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k,:,:,0],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label
def get_test_batch(batch_size):
    j = np.random.randint(20788,22424,batch_size)
    batch = np.zeros((batch_size,h,w,1))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k,:,:,0],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label

In [ ]:
num_steps = 500000000
lr = 0.0001
display_step = 200
batch_size = 64

In [ ]:
X = tf.placeholder (dtype=tf.float32,shape=[None, h,w,1])

con0 = tf.layers.conv2d(inputs=X,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
con1 = tf.layers.conv2d(inputs=con0,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
max1 = tf.layers.max_pooling2d(inputs=con1,pool_size=2,strides=2)
#######
con2 = tf.layers.conv2d(inputs=max1,kernel_size=(3,3),filters=32,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
con3 = tf.layers.conv2d(inputs=con2,kernel_size=(3,3),filters=32,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
max3 = tf.layers.max_pooling2d(inputs=con3,pool_size=2,strides=2)

con4 = tf.layers.conv2d(inputs=max3,kernel_size=(3,3),filters=64,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
con5 = tf.layers.conv2d(inputs=con4,kernel_size=(3,3),filters=64,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
max5 = tf.layers.max_pooling2d(inputs=con5,pool_size=2,strides=2)

con6 = tf.layers.conv2d(inputs=max5,kernel_size=(3,3),filters=64,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
con7 = tf.layers.conv2d(inputs=con6,kernel_size=(3,3),filters=64,activation=tf.nn.relu,strides=1,padding='same',kernel_initializer=tf.variance_scaling_initializer)
max7 = tf.layers.max_pooling2d(inputs=con7,pool_size=2,strides=2)

hidden_layer = tf.reshape(max7,[-1,48*64])
y_1 = tf.layers.dense(inputs=hidden_layer,units=5000,activation=tf.nn.relu,kernel_initializer=tf.variance_scaling_initializer)

out = tf.layers.dense(inputs=y_1,units=w*h)
x = tf.reshape(X,shape=[-1,h*w])
#loss = tf.reduce_mean(tf.pow(x - out, 2))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=out))
Y = tf.placeholder (dtype=tf.float32,shape=[None, 10])
y_11 = tf.layers.dense(inputs=y_1,units=1000,activation=tf.nn.relu,kernel_initializer=tf.variance_scaling_initializer)
y_2 = tf.layers.dense(inputs=y_11,units=10)
clf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_2))
correctPred = tf.equal(tf.argmax(y_2,1), tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
lrr = tf.placeholder(shape=[],dtype=tf.float32)
optimizer = tf.train.AdamOptimizer(learning_rate=lrr).minimize(clf_loss)
optimizer2 =  tf.train.AdamOptimizer(learning_rate=lrr).minimize(loss)

##########################################################




In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
clf_l = []
l_l = []
learn = 0.001

In [ ]:
for i in range(1,10001):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x,batch_y = get_batch(64)

    # Run optimization op (backprop) and cost op (to get loss value)
    if i % 2 == 3:
        _, l = sess.run([optimizer, clf_loss], feed_dict={X: batch_x,Y:batch_y,lrr:0.001})
        print('Class:',l)
        clf_l.append(l)
    else:
        _, l = sess.run([optimizer2, loss], feed_dict={X: batch_x,Y:batch_y,lrr:learn})
        print('Recons:',l)
        l_l.append(l)
    if i % 10 == 1: 
        learn *= 0.9
        total_acc  =  0
        batch_x,batch_y = get_test_batch(16)
        img = sess.run(out, feed_dict={X: batch_x,Y:batch_y})
        plt.imshow(img[0].reshape(h,w),'gray')
        plt.show()
total_acc  =  0
for i in range(100):
        batch_x,batch_y = get_test_batch(16)
        l,acc = sess.run([clf_loss,accuracy], feed_dict={X: batch_x,Y:batch_y})
        total_acc += acc
print('-----------------------------Validation:',total_acc/100)
plt.semilogy(l_l,label='Recon Error')
plt.semilogy(clf_l,label='Class. Error')
plt.legend()
plt.show()
img = sess.run(out, feed_dict={X: batch_x,Y:batch_y})
plt.imshow(img[0].reshape(h,w),'gray')

In [ ]:
total_acc  =  0
for i in range(100):
        batch_x,batch_y = get_test_batch(16)
        l,acc = sess.run([clf_loss,accuracy], feed_dict={X: batch_x,Y:batch_y})
        total_acc += acc
print('-----------------------------Validation:',total_acc/100)
plt.semilogy(l_l[0:1100],label='Recon Error')
plt.semilogy(clf_l[0:1100],label='Class. Error')
plt.legend()
plt.show()
img = sess.run(out, feed_dict={X: batch_x,Y:batch_y})
plt.imshow(img[0].reshape(h,w),'gray')

In [ ]:
learn  = 0.00001